In [1]:
import os
import folium
import json
import branca.colormap as cm
import pandas as pd
import requests

# GeoJSON and choropleth


## Using `GeoJson`


Let us load a GeoJSON file

### voor stadsdelen:
http://maps.amsterdam.nl/open_geodata/geojson.php?KAARTLAAG=GEBIED_STADSDELEN&THEMA=gebiedsindeling
### voor buurten:
http://maps.amsterdam.nl/open_geodata/geojson.php?KAARTLAAG=GEBIED_BUURTEN&THEMA=gebiedsindeling
### voor wijken:
http://maps.amsterdam.nl/open_geodata/geojson.php?KAARTLAAG=GEBIED_BUURTCOMBINATIES&THEMA=gebiedsindeling
### voor gebieden:
http://maps.amsterdam.nl/open_geodata/geojson.php?KAARTLAAG=GEBIEDEN22&THEMA=gebiedsindeling

In [2]:
response = requests.get('http://maps.amsterdam.nl/open_geodata/geojson.php?KAARTLAAG=GEBIED_BUURTCOMBINATIES&THEMA=gebiedsindeling')
geo_json_data = response.json()

Laad hieronder je pandas dataframe in. Dit dataframe moet tenminste "ID" en "Value" bevatten. 
- ID geeft de code van de gebiedsaanduiding aan (bv. "E14" voor "Staatsliedenbuurt") 
- Value geeft een waarde aan.

Hieronder maken we gebruik van een test dataframe:

In [3]:
import numpy as np

test_df = pd.read_csv('http://maps.amsterdam.nl/open_geodata/excel.php?KAARTLAAG=GEBIED_BUURTCOMBINATIES&THEMA=gebiedsindeling', delimiter=';')
test_df["ID"] = test_df["Buurtcombinatie_code"]
test_df = test_df[["ID"]]

test_df["Value"] =  np.random.randint(0, 100, test_df.shape[0])
test_df.head(5)

,ID,Value
0,A00,22
1,A01,25
2,A02,49
3,A03,46
4,A04,16


Now we need to create a function that maps one value to a RGB color (of the form `#RRGGBB`).
For this, we'll use colormap tools from `folium.colormap`.

In [4]:
linear = cm.LinearColormap(['#E5F2FC','#B1D9F5','#71BDEE','#00a0e6','#004699'])
linear

colormap = linear.scale(
    test_df.Value.min(),
    test_df.Value.max())

Door een index toe te voegen kun je de schaal van de kleuren aanpassen; bijvoorbeeld dat de kleur alleen donker is bij hele hoge of hele lage waarden. Let wel op: kleuren vallen weg als de schaal niet overeenkomt met je min en max waarde. Ook moeten het aantal punten dat je specificeert in je index overeenkomen met het aantal kleuren. 

In [5]:
#colormap.index =[0, 0.01, 0.02,0.025, 0.072]
print(colormap(5.0))

colormap

#dbedfb


In [6]:
test_dict = test_df.set_index('ID')['Value']

m = folium.Map(location=[52.379189, 4.899431], tiles='https://{s}.data.amsterdam.nl/topo_google/{z}/{x}/{y}.png',    
                attr='Amsterdam', zoom_start=13, min_lat=52.269470,
               max_lat=52.4322, min_lon=4.72876, max_lon=5.07916, subdomains=['t1', 't2', 't3', 't4'])

folium.GeoJson(geo_json_data, style_function=lambda feature: {
        'fillColor': colormap(test_dict[feature['properties']['Buurtcombinatie_code']]),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5',
        'fillOpacity': 0.9
    }).add_to(m)
        
colormap.caption = 'Color scale'
colormap.add_to(m)

m.save('GeoJSON_and_choropleth_7_Amsterdam.html')
m


Hieronder dezelfde kaart, maar dan als functie:

In [30]:
import re
import os
import folium
import json
import branca.colormap as cm
import pandas as pd
import requests

def create_map(df, geotype, output_name):
    """
    Input: 
        - df, contains at least:
            - ID (with the code of stadsdeel, gebied, buurtcombinatie or buurt)
            - Value
        - geotype (string):
            - stadsdeel 
            - gebied
            - buurtcombinatie
            - buurt
        - output_name (string): how do you want te file to be saved
    Ouput: html with data on map
    
    """
    if geotype not in ['buurtcombinatie', 'gebied', 'buurt', 'stadsdeel']:
        print("geotype wordt niet ondersteund. Kies een vande volgende opties: 'buurtcombinatie', 'gebied', 'buurt' of 'stadsdeel'.")
        return

    if bool(re.search(r"\W", output_name)):
        print("ongeldige outputnaam, outputnaam mag alleen letters, cijfers of _ bevatten")
        return
    
    if not "ID" in df.columns or not "Value" in df.columns:
        print("dataframe moet 'ID' en 'Value' bevatten")
        return
    
    if geotype == 'buurtcombinatie':
        response = requests.get('http://maps.amsterdam.nl/open_geodata/geojson.php?KAARTLAAG=GEBIED_BUURTCOMBINATIES&THEMA=gebiedsindeling')
        geo_json_data = response.json()
    elif geotype == 'gebied':
        response = requests.get('http://maps.amsterdam.nl/open_geodata/geojson.php?KAARTLAAG=GEBIEDEN22&THEMA=gebiedsindeling')
        geo_json_data = response.json()
    elif geotype == 'buurt':
        response = requests.get('http://maps.amsterdam.nl/open_geodata/geojson.php?KAARTLAAG=GEBIED_BUURTEN&THEMA=gebiedsindeling')
        geo_json_data = response.json()
    elif geotype == 'stadsdeel':
        response = requests.get('http://maps.amsterdam.nl/open_geodata/geojson.php?KAARTLAAG=GEBIED_STADSDELEN&THEMA=gebiedsindeling')
        geo_json_data = response.json()
    
    #set colormap
    linear = cm.LinearColormap(['#E5F2FC','#B1D9F5','#71BDEE','#00a0e6','#004699'])

    colormap = linear.scale(
        df.Value.min(),
        df.Value.max())
    
    #convert df to dict
    df_dict = df.set_index('ID')['Value']
    
    #make map
    m = folium.Map(location=[52.379189, 4.899431], tiles='https://{s}.data.amsterdam.nl/topo_google/{z}/{x}/{y}.png',    
                attr='Amsterdam', zoom_start=13, min_lat=52.269470,
               max_lat=52.4322, min_lon=4.72876, max_lon=5.07916, subdomains=['t1', 't2', 't3', 't4'])
    
    #plot data on map
    folium.GeoJson(geo_json_data, style_function=lambda feature: {
        'fillColor': colormap(df_dict[feature['properties']['Buurtcombinatie_code']]),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5',
        'fillOpacity': 0.9
        }).add_to(m)
    
    #add scale to map
    colormap.caption = 'Color scale'
    colormap.add_to(m)
    
    #save map
    m.save('%s.html' %output_name)
    m
    print("kaart is succesvol opgeslagen")
    

Test of je input werkt: 

In [31]:
create_map(test_df, "buurtcombinatie", "dit_is_een_test")

kaart is succesvol opgeslagen
